In [1]:
import pandas as pd

In [2]:
pd.read_json("sentences.json")

,0
0,A pandemic is an epidemic of an infectious dis...
1,The most fatal pandemic in recorded history wa...
2,Current pandemics include COVID-19 (SARS-CoV-2...
3,"As of 2018, approximately 37.9 million people ..."
4,Cholera is an infection of the small intestine...
5,Classic cholera symptom is large amounts of wa...
6,"The COVID-19 pandemic, also known as the coron..."
7,"Common symptoms of COVID-19 include fever, cou..."
8,The Plague of Cyprian was a pandemic that affl...
9,"The Spanish flu, also known as the 1918 flu pa..."


Use AutoTokenizer and AutoModel classes from Transformers library to load a pre-trained model from Transformers, along with the appropriate tokenizer.
https://huggingface.co/docs/transformers/model_doc/auto